In [ ]:
from nbpypath import *

In [ ]:
import numpy as np
import pandas as pd

from numba import njit


In [ ]:
from oscb import *

# customize Bokeh plotting in the Jupyter Notebook environment
setup_notebook()

In [ ]:
# Seed the PRNG
random_seed(12345)

In [ ]:
# create an lnet, it initially has no synaptic connection at all
# with each letter having 10 columns, 5x that number of total columns,
# and 100 cells per column, there are 130K neurons total.
#
# if all neurons are fully connected, there'll be 16.9B synapses/parameters.
#
# a LetterNet will hard-cap the number of synapses, 1M by default,
# it'll drop weakest synapses out, once the total number reaches 1M, a
# LOAD_FACTOR=0.8 configues that each compaction (synapse dropout) should
# retain 0.8M synapses.
#
lnet = LetterNet(
    MAX_SYNAPSES=1_000_000,  # max number of synapses, one million by default
    N_COLS_PER_LETTER=10,  # distributedness of letter SDR
    SPARSE_FACTOR=5,  # sparseness of letter SDR
    N_CELLS_PER_COL=100,  # per mini-column capacity
)

In [ ]:
%%time

lnet.learn_words_as_sequence(
    [], # no input data to see JIT time cost
    sp_width=(1, 1),  # width of spike train: [n_columns, n_cells]
    sp_thick=1,  # thickness of spike train
)


In [ ]:
%%time

lnet.learn_words_as_sequence(
    '''
the quick brown fox jumped over the old lazy dog
''',
    sp_width=(3, 20),  # width of spike train: [n_columns, n_cells]
    sp_thick=60,  # thickness of spike train
)


In [ ]:
# get a DataFrame about all excitatory synapses so far
excit_df = lnet.excitatory_synapses()
excit_df

In [ ]:
# statistics of individual synapses
excit_df.efficacy.describe()

In [ ]:
# statistics of all incoming synapses per individual postsynaptic neurons
excit_df.groupby(['to_column', 'to_cell'])['efficacy'].sum().describe()

In [ ]:
lsim = LetterNetSim(
    lnet,

    # global scaling factor, to accommodate a unit synaptic efficacy value of 1.0
    # roughly this specifies that:
    #   how many presynaptic spikes is enough to trigger a postsynaptic spike,
    #   when each synapse has a unit efficacy value of 1.0
    SYNAP_FACTOR=30,

    # reset voltage, equal to VOLT_REST, or lower to enable refractory period
    VOLT_RESET=-0.1,
    # membrane time constant
    τ_m=10,

    # fire plot params
    plot_width=800,
    plot_height=600,
    plot_n_steps=100,
    fire_dots_glyph="square",
    fire_dots_alpha=0.01,
    fire_dots_size=3,
    fire_dots_color="#0000FF",
)

lsim.simulate(
    50, # n steps

    '''
ow
''',
    prompt_blur=0.0,  # entirely suppress other signals
)

lsim.simulate(
    50, # n steps

    '''
ow
''',
    prompt_blur=0.5,  # half other signals
)

# ref the learnt seq:
#   the quick brown fox jumped over the old lazy dog
show(lsim.fig)